In [36]:
!pip install tifffile
!pip install opencv-python
!pip install rasterio
!pip install scipy
!pip install torchvision
!pip install torchmetrics

   ---------------------------------------- 0.0/840.4 kB ? eta -:--:--
   ---------------------------------------- 10.2/840.4 kB ? eta -:--:--
   -------- ------------------------------- 174.1/840.4 kB 2.6 MB/s eta 0:00:01
   ----------------------- ---------------- 491.5/840.4 kB 4.4 MB/s eta 0:00:01
   ---------------------------------------  839.7/840.4 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 840.4/840.4 kB 4.9 MB/s eta 0:00:00


In [1]:
!pip install --upgrade scikit-image

In [1]:
import numpy as np
import pandas as pd
from skimage import io, img_as_float
import os
import tifffile
import torch 
import imageio
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import rasterio
from rasterio.plot import show
from PIL import Image
from torchvision.transforms import ToTensor
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets

In [1]:
pip install --upgrade mkl

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Load in data with PyTorch (switched train and test cause test is bigger)
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

# Load train set
train_dataset = datasets.ImageFolder(root='test/organized', transform=transform)

# Load test set
test_dataset = datasets.ImageFolder(root='train/organized', transform=transform)

In [3]:
print(f"Train data:\n{train_dataset}\nTest data:\n{test_dataset}")

Train data:
Dataset ImageFolder
    Number of datapoints: 8675
    Root location: test/organized
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               Grayscale(num_output_channels=1)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 6950
    Root location: train/organized
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               Grayscale(num_output_channels=1)
               ToTensor()
           )


In [4]:
os.cpu_count()

4

In [4]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=10, # how many samples per batch?
                              num_workers=os.cpu_count(), # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_dataset, 
                             batch_size=10, 
                             num_workers=os.cpu_count(), 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1a05f9faf90>,
 <torch.utils.data.dataloader.DataLoader at 0x1a05fa60dd0>)

In [5]:
# Create a convolutional neural network 
from torch import nn

class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 56 * 56,  # Adjusted input size
                      out_features=output_shape),
            nn.Sigmoid()  # Sigmoid activation for binary classification
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x.view(x.size(0), -1))
        return x

torch.manual_seed(42)
model_2 = TinyVGG(input_shape=1, hidden_units=10, output_shape=1)
model_2


TinyVGG(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=31360, out_features=1, bias=True)
    (2): Sigmoid()
  )
)

In [12]:
# Setup loss and optimizer
from torch.nn import BCEWithLogitsLoss

loss_fn = loss_fn = BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), 
                             lr=1)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [13]:
from torchmetrics.classification import BinaryStatScores

def train_step(model, data_loader, loss_fn, optimizer, accuracy_fn):
    train_loss = 0
    binary_stat_scores = BinaryStatScores()

    model.train()
    for X, y in data_loader:
        y_pred = torch.sigmoid(model(X))
        loss = loss_fn(y_pred, y.unsqueeze(1).float())  # Keep unsqueeze for binary classification
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update binary statistics
        binary_stat_scores.update(y_pred, y.unsqueeze(1).float())

    # Calculate accuracy per epoch and print out
    metrics = binary_stat_scores.compute()
    print(metrics)

def test_step(data_loader, model, loss_fn, accuracy_fn):
    test_loss = 0
    binary_stat_scores = BinaryStatScores()

    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # 1. Forward pass with sigmoid activation for binary classification
            test_pred = torch.sigmoid(model(X))

            # 2. Calculate loss using binary cross-entropy loss
            loss = loss_fn(test_pred, y.unsqueeze(1).float())  # Keep unsqueeze for binary classification
            test_loss += loss.item()

            # Update binary statistics
            binary_stat_scores.update(test_pred, y.unsqueeze(1).float())

    # Calculate accuracy and print out
    metrics = binary_stat_scores.compute()
    print(metrics)


In [14]:
#Train TinyVGG
from tqdm.auto import tqdm
from torchmetrics.classification import BinaryAccuracy

# Measure time
from timeit import default_timer as timer
train_time_start_model_2 = timer()

# Train and test model 
epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader, 
        model=model_2, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn)
    
    test_step(data_loader=test_dataloader,
        model=model_2,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn)

train_time_end_model_2 = timer()
print(train_time_end_model_2 - train_time_start_model_2)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
tensor([ 614, 8061,    0,    0,  614])
tensor([ 358, 6592,    0,    0,  358])
Epoch: 1
---------
tensor([ 614, 8061,    0,    0,  614])
tensor([ 358, 6592,    0,    0,  358])
Epoch: 2
---------
tensor([ 614, 8061,    0,    0,  614])
tensor([ 358, 6592,    0,    0,  358])
1384.800298599992


In [ ]:
#Transfer learn full VGG16 instead
